In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from torch.utils.data import Dataset,DataLoader
from transformers import AdamW

In [ ]:
!pip install pyarrow datasets

In [5]:
from datasets import load_dataset

Question_Generation = load_dataset("christti/squad-augmented-v2")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
print(Question_Generation.keys())

dict_keys(['train', 'validation'])


In [7]:
Question_Generation.shape

{'train': (169211, 5), 'validation': (10570, 5)}

In [8]:
print(Question_Generation['train'][0])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


In [9]:
train_data=pd.DataFrame(Question_Generation['train']).sample(frac=0.5)
validation_data=pd.DataFrame(Question_Generation['validation']).sample(frac=0.5)

In [10]:
train_data.shape

(84606, 5)

In [11]:
validation_data.shape

(5285, 5)

In [12]:
train_data.head()

,id,title,context,question,answers
39309,5726f25ff1498d1400e8f0b5,Napoleon,Critics argue Napoleon's true legacy must refl...,How many years of wars does Victor Davis Hanso...,"{'text': ['17'], 'answer_start': [207]}"
61328,5727f2e03acd2414000df091,Gamal_Abdel_Nasser,The Egyptian singer Umm Kulthum hosted a publi...,What Egyptian singer performed at the return o...,"{'text': ['Umm Kulthum'], 'answer_start': [20]}"
123224,572631d1ec44d21400f3dc00_aug,Freemasonry,"Although Freemasonry is not a religion, a Prot...",What Grand Lodge had a withdrawal of recogniti...,"{'text': ['Grand Orient of France'], 'answer_s..."
35713,571aab8910f8ca14003052b9,Athanasius_of_Alexandria,"T. Gilmartin, (Professor of History, Maynooth,...",What was the belief that was approved at this ...,"{'text': ['The Son to be ""like the Father""'], ..."
146638,572831fa2ca10214002da05f_aug,Annelid,Accounts of the use of leeches for the medical...,When did India begin using leeches?,"{'text': ['200 AD'], 'answer_start': [134]}"


In [13]:
#train_data=train_data.drop(["id","title"],axis=1)
train_data=validation_data.drop(["id","title","answers"],axis=1)
validation_data=validation_data.drop(["id","title","answers"],axis=1)

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5285 entries, 1590 to 3919
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   5285 non-null   object
 1   question  5285 non-null   object
dtypes: object(2)
memory usage: 123.9+ KB


In [15]:
# تحويل البيانات إلى قوائم نصوص
train_input = train_data[['context', 'question']].apply(lambda x: x['context'] + " " + x['question'], axis=1).tolist()
train_output = train_data['question'].tolist()  # السؤال فقط هو المخرج

validation_input = validation_data[['context', 'question']].apply(lambda x: x['context'] + " " + x['question'], axis=1).tolist()
validation_output = validation_data['question'].tolist()  # السؤال فقط هو المخرج

In [16]:
train_input

["Tesla obtained around 300 patents worldwide for his inventions. Some of Tesla's patents are not accounted for, and various sources have discovered some that have lain hidden in patent archives. There are a minimum of 278 patents issued to Tesla in 26 countries that have been accounted for. Many of Tesla's patents were in the United States, Britain, and Canada, but many other patents were approved in countries around the globe.:62 Many inventions developed by Tesla were not put into patent protection. In how many countries did Tesla hold patents?",
 'A consortium led by the International Crops Research Institute for the Semi-Arid Tropics (ICRISAT) has had some success in helping farmers grow new pigeon pea varieties, instead of maize, in particularly dry areas. Pigeon peas are very drought resistant, so can be grown in areas with less than 650 mm annual rainfall. Successive projects encouraged the commercialisation of legumes, by stimulating the growth of local seed production and agr

In [17]:
train_output

['In how many countries did Tesla hold patents?',
 'Why did they teach them to grow peas?',
 'What is generated between a surface and an object that is being pushed?',
 'When did the German army reoccupy Rhineland?',
 'What causes Pauli repulsion?',
 'What company was chosen to build an AC distribution system at Niagara Falls?',
 'How many hymns did Luther write for the first choral hymnal?',
 'What does pharmacy legislation mandate?',
 'Who is VideoGuard owned by?',
 'In what year did the FCC begin an investigation in to the operation of radio networks in America',
 "What type of flower is sought on Midsummer's Eve?",
 'In what years did Doctor Who originally show on TV?',
 'In what century was quantum mechanics made? ',
 'What was the name of the experiment that tested how life originated?',
 'Who directed Stormy Monday?',
 'What unintended development did the relationship with the elector have on church government?',
 'What formation has an asymmetrical pattern of different terraces

`strat the model`




using model (google/pegasus-large)

In [18]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [19]:
def encode_data(inputs, targets, tokenizer, max_length=512):
    # تعيين رمز padding
    tokenizer.pad_token = tokenizer.eos_token

    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return input_encodings, target_encodings

train_input_encodings, train_target_encodings = encode_data(train_input, train_output, tokenizer)
validation_input_encodings, validation_target_encodings = encode_data(validation_input, validation_output, tokenizer)


In [20]:
from torch.utils.data import Dataset, DataLoader
import torch

# تعريف Dataset لتغذية البيانات للنموذج
class QGDataset(Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings
        
        # التأكد من أن الأحجام متساوية
        assert len(self.input_encodings.input_ids) == len(self.target_encodings.input_ids), \
            "Input and target encodings must be the same length"

    def __len__(self):
        return len(self.input_encodings.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_encodings.input_ids[idx]),
            'attention_mask': torch.tensor(self.input_encodings.attention_mask[idx]),
            'labels': torch.tensor(self.target_encodings.input_ids[idx])  # labels should be input_ids of the target
        }

# إعداد Datasets
train_dataset = QGDataset(train_input_encodings, train_target_encodings)
validation_dataset = QGDataset(validation_input_encodings, validation_target_encodings)

# إعداد DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8)


In [21]:
# إعداد GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# # إعداد معايير التدريب
# training_args = TrainingArguments(
#     output_dir='./results',               # مكان حفظ النموذج المدرب
#     evaluation_strategy="epoch",          # التقييم بعد كل حقبة
#     learning_rate=2e-5,                   # معدل التعلم
#     per_device_train_batch_size=8,        # حجم الدفعة للتدريب
#     per_device_eval_batch_size=8,         # حجم الدفعة للتقييم
#     num_train_epochs=25,                   # عدد الحقبات
#     weight_decay=0.01,                    # التخفيف من الوزن
#     logging_dir='./logs',                  # مكان حفظ السجلات
# )

# # إنشاء Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=validation_dataset,
# )

# # بدء التدريب
# trainer.train()



T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [23]:
# إعداد TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # استخدم eval_strategy بدلاً من evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="none",  # تعطيل W&B
)

# إعداد Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

# بدء التدريب
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_36/1786636006.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.input_encodings.input_ids[idx]),
/tmp/ipykernel_36/1786636006.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.input_encodings.attention_mask[idx]),
/tmp/ipykernel_36/1786636006.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach

Epoch,Training Loss,Validation Loss
1,No log,0.004358
2,0.006200,0.003819
3,0.006200,0.003619
4,0.005600,0.003316
5,0.005100,0.003151
6,0.005100,0.003010
7,0.004300,0.002896
8,0.004300,0.002789
9,0.004300,0.002739
10,0.003700,0.002717


/tmp/ipykernel_36/1786636006.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(self.input_encodings.input_ids[idx]),
/tmp/ipykernel_36/1786636006.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(self.input_encodings.attention_mask[idx]),
/tmp/ipykernel_36/1786636006.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.target_encodings.input_ids[idx])  # labels should be input_ids of the target
/opt/conda/lib/python3.10/site-packages/torch/nn/parall

TrainOutput(global_step=3310, training_loss=0.004782202560735973, metrics={'train_runtime': 1602.4899, 'train_samples_per_second': 32.98, 'train_steps_per_second': 2.066, 'total_flos': 7152814207795200.0, 'train_loss': 0.004782202560735973, 'epoch': 10.0})

In [26]:
# حفظ النموذج المدرب
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/spiece.model',
 './saved_model/added_tokens.json')

In [32]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# تحميل النموذج والمُرمّز
model = T5ForConditionalGeneration.from_pretrained('./saved_model')
tokenizer = T5Tokenizer.from_pretrained('./saved_model')

# إعداد GPU إذا كان متاحًا
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# دالة لتوليد الأسئلة من نص
def generate_question(context):
    # ترميز المدخلات
    input_ids = tokenizer.encode(context, return_tensors='pt').to(device)
    
    # توليد السؤال
    output_ids = model.generate(input_ids, max_length=30, num_beams=5, early_stopping=True)
    
    # تحويل الإخراج إلى نص
    question = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return question

# تجربة نصوص مختلفة للاختبار
# نص كبير للاختبار
large_context = """


The Impact of Climate Change on Global Ecosystems

Climate change is one of the most pressing issues facing our world today. It refers to long-term changes in temperatures and weather patterns, largely due to human activities such as burning fossil fuels and deforestation. These activities increase the concentration of greenhouse gases in the atmosphere, which trap heat and lead to global warming. The impacts of climate change are widespread, affecting ecosystems, human health, and the global economy.

One of the most significant impacts of climate change is the disruption of ecosystems. Rising temperatures affect species' habitats, forcing animals to migrate or adapt to new conditions. For example, polar bears are struggling as the ice caps in the Arctic melt, reducing their hunting grounds. Similarly, coral reefs are experiencing bleaching events due to warmer ocean temperatures, which threaten marine biodiversity.

Changes in precipitation patterns are another consequence of climate change. Some regions are experiencing more intense and frequent droughts, while others face increased flooding. These shifts in weather patterns can devastate agriculture, leading to food shortages and threatening livelihoods, particularly in developing countries. Furthermore, extreme weather events, such as hurricanes, heatwaves, and wildfires, are becoming more frequent and severe, posing additional risks to both human populations and natural ecosystems.

Climate change also exacerbates existing social inequalities. Vulnerable communities, particularly in low-income countries, are disproportionately affected by climate-related disasters. These populations often lack the resources and infrastructure to adapt to the changes, leading to displacement, loss of livelihoods, and increased poverty. Additionally, climate change has significant health impacts, including heat-related illnesses, the spread of infectious diseases, and respiratory problems caused by air pollution.

Global cooperation is essential to mitigate the effects of climate change. Governments, businesses, and individuals all play a role in reducing greenhouse gas emissions and transitioning to sustainable energy sources. International agreements, such as the Paris Agreement, aim to limit global warming to below 2°C above pre-industrial levels. However, more ambitious efforts are needed to prevent the worst consequences of climate change.

In conclusion, climate change is a global crisis that requires immediate action. Its impacts on ecosystems, human health, and social structures are far-reaching and severe. By working together, we can protect our planet and ensure a sustainable future for generations to come.

"""

# اختبار النموذج على النص الكبير
generated_question_large = generate_question(large_context)
print(f"Generated Question: {generated_question_large}")



Generated Question: <pad> By working together, we can protect our planet and ensure a sustainable future for generations to come.
